In [2]:
%load_ext autoreload
%autoreload 2

import random
import time

import matplotlib.pyplot as plt
import torch
from torch import nn

from tbfm import tbfm, tbfm_phase_invariant
from tbfm.bases import FourierBases
from tbfm.test import WaveDataGenerator, plot_grid_activity
from tbfm.utils import fft_circular_shift_per_basis



BATCH_SIZE = 5000
RUNWAY_LENGTH = 20
TRIAL_LENGTH = 184
N_DEPTH = 6
TRAIN_SET_SIZE = 3000
DEPTH_RUNWAY = 5
STIM_DIM = N_DEPTH * DEPTH_RUNWAY
DEVICE = 'cuda'

In [3]:
train_runways, train_y, train_theta, test_runways, test_y, test_theta = torch.load('blah.torch')

In [5]:
# Seems to work. Let's try the shift invariant TBFM.
NUM_BASES = 20
LATENT_DIM = 6
BASIS_DEPTH = 4

num_channels = train_runways.shape[2]
model = tbfm_phase_invariant.TBFMPi(num_channels, STIM_DIM, RUNWAY_LENGTH, NUM_BASES,
                  TRIAL_LENGTH-RUNWAY_LENGTH,
                  batchy=train_y,
                  latent_dim=LATENT_DIM,
                  basis_depth=BASIS_DEPTH,
                  device=DEVICE)
optim = model.get_optim(lr=2e-4)


In [6]:
NUM_EPOCHS = 20000
TEST_EPOCH_INTERVAL = 1000  # We will run against the test set every so often to monitor progress
LAMBDA = 0.01

losses_train = []
losses_test = []

# Now z score our y values using the comming means/stdevs from the training set.
# Runways are z scored by the model on the fly, and y_hats will be in that z-scored space
y_train = model.zscore(train_y)
y_test = model.zscore(test_y)
start_time = time.time()
for eidx in range(NUM_EPOCHS):
    optim.zero_grad()
    
    yhat_train = model(train_runways, train_theta)
    
    # Loss will be based on surface activity predictions only.
    loss = nn.MSELoss()(yhat_train, y_train) 

    l1w = model.get_weighting_reg()
    (loss + LAMBDA * l1w).backward()
          
    losses_train.append((eidx, loss.item()))
    optim.step()
    
    if (eidx % TEST_EPOCH_INTERVAL) == 0:        
        with torch.no_grad():
            yhat_test = model(test_runways, test_theta)
    
            loss = nn.MSELoss()(yhat_test, y_test)
            losses_test.append((eidx, loss.item()))
            
        print(f"epoch: {eidx}, train loss: {losses_train[-1][1]}, test loss: {losses_test[-1][1]}")

print(f"Training time: {time.time() - start_time}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.35 GiB. GPU 0 has a total capacity of 7.65 GiB of which 1.22 GiB is free. Process 85104 has 846.00 MiB memory in use. Including non-PyTorch memory, this process has 5.58 GiB memory in use. Of the allocated memory 5.39 GiB is allocated by PyTorch, and 31.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)